In [1]:
import geopandas as gpd
from shapely.geometry import box
from pyproj import CRS
import pandas as pd
import numpy as np
from math import cos, pi
from pyproj import Transformer


In [2]:
# 加载上海市边界
shanghai = gpd.read_file("./data/Shanghai-2020/Shanghai-2020.shp")

# 首先设置原始数据的CRS
shanghai = shanghai.set_crs("EPSG:4326")

# # 然后转换为UTM投影（上海位于UTM zone 51N）
# shanghai_utm = shanghai.to_crs("EPSG:32651")

# 获取边界范围
bounds = shanghai.total_bounds
minx, miny, maxx, maxy = bounds

# # 网格大小（1km）
# grid_size = 1000  # 单位：米

# 网格大小（1km，转换为经纬度）
# 1度纬度 ≈ 111km，1度经度 ≈ 111km * cos(latitude)
# 计算中心纬度
lat_center = (miny + maxy) / 2
grid_size_lat = 1 / 111  # 纬度方向
grid_size_lon = 1 / (111 * cos(lat_center * pi / 180))  # 经度方向

# # 创建网格
# grid_cells = []
# x = minx
# while x < maxx:
#     y = miny
#     while y < maxy:
#         grid_cells.append(box(x, y, x + grid_size, y + grid_size))
#         y += grid_size
#     x += grid_size

# 创建网格
grid_cells = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        grid_cells.append(box(x, y, x + grid_size_lon, y + grid_size_lat))
        y += grid_size_lat
    x += grid_size_lon

# 转换为GeoDataFrame
grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs="EPSG:4326")

# 裁剪网格（只保留上海市范围内的网格）
clipped_grid = gpd.clip(grid, shanghai)

# 将几何对象转换为最小外接矩形
clipped_grid['geometry'] = clipped_grid.geometry.envelope

In [3]:
# 转换为UTM投影进行精确面积计算
temp_grid = clipped_grid.to_crs("EPSG:32651")

# 创建转换器
transformer = Transformer.from_crs("EPSG:32651", "EPSG:4326", always_xy=True)

# 转换为Pandas DataFrame
grid_data = []

for idx, row in clipped_grid.iterrows():
    coords = list(row.geometry.exterior.coords)
    grid_data.append({
        'grid_id': idx,
        'min_x': coords[0][0],
        'min_y': coords[0][1],
        'max_x': coords[2][0],
        'max_y': coords[2][1],
        'center_x': row.geometry.centroid.x,
        'center_y': row.geometry.centroid.y
    })

# 创建DataFrame
grid_df = pd.DataFrame(grid_data)

# 保存为CSV
grid_df.to_csv("./data/grid/shanghai_grid_wgs84.csv", index=False)

In [4]:
import folium
from pyproj import Transformer

# # 创建坐标转换器
# transformer = Transformer.from_crs("EPSG:32651", "EPSG:4326", always_xy=True)

# # 转换中心点坐标
# grid_df['center_lon'], grid_df['center_lat'] = transformer.transform(
#     grid_df['center_x'], grid_df['center_y']
# )

# 创建地图（以人民广场为中心）
m = folium.Map(location=[31.23, 121.47], zoom_start=10)

# # 添加网格
# for _, row in grid_df.iterrows():
#     # 转换四个角点坐标
#     coords = [
#         transformer.transform(row['min_x'], row['min_y']),
#         transformer.transform(row['max_x'], row['max_y'])
#     ]
    
#     # 添加矩形
#     folium.Rectangle(
#         bounds=[
#             [coords[0][1], coords[0][0]],  # 左下角 (lat, lon)
#             [coords[1][1], coords[1][0]]   # 右上角 (lat, lon)
#         ],
#         color='blue',
#         fill=False,
#         weight=1
#     ).add_to(m)

# 添加网格
for _, row in clipped_grid.iterrows():
    coords = list(row.geometry.exterior.coords)
    folium.Rectangle(
        bounds=[
            [coords[0][1], coords[0][0]],  # 左下角
            [coords[2][1], coords[2][0]]   # 右上角
        ],
        color='blue',
        fill=False
    ).add_to(m)

# 显示地图
m.save('shanghai_grid.html')